## Object detection and object counting in polygon zone: video file annotation

This notebook is an example how to use DeGirum PySDK to do object detection and object 
counting in polygon zone, annotating video file.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want run inference, video file name, model name, and other options here

In [ ]:
# hw_location: where you want to run inference
#     @cloud to use DeGirum cloud
#     @local to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local web camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# polygon_zones: zones in which objects need to be counted
# class_list: list of 
hw_location='@cloud'
model_zoo_url = 'https://cs.degirum.com/degirum/public'
model_name= 'yolo_v5s_coco--512x512_quant_n2x_orca1_1'
video_source = '../../images/example_video.mp4'        
polygon_zones = [[[30, 600], [300, 480], [640, 480], [700, 600]],
                 [[880, 600], [750, 480], [1030, 480], [1280, 600]],
                ]
class_list=["car", "motorbike", "truck"]

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, degirum_tools
# configure for Google Colab
degirum_tools.configure_colab()  
# connect to AI inference engine getting token from env.ini file
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
import IPython
from pathlib import Path

orig_path = Path(video_source)
ann_path = "../../workarea/" + orig_path.stem + "_annotated" + orig_path.suffix

# load model
model = zoo.load_model(model_name, overlay_line_width=1)
# create zone counter
zone_counter = degirum_tools.ZoneCounter(polygon_zones,
                                         class_list=class_list,
                                         triggering_position=degirum_tools.ZoneCounter.CENTER,
                                         )
# annotate video
degirum_tools.annotate_video(model, orig_path, ann_path, zone_counter=zone_counter)

In [ ]:
# display result
IPython.display.Video(ann_path, embed=degirum_tools._in_colab())